# AD20 Milestone 1
Group 20: Lindsey Brown, Xinyue Wang, Kevin Yoon

# Table of Contents

**1. Introduction**

    1.1 Automatic Differentiation as a Solution to the Problem of Computing Derivatives
    1.2 Significance of AD Techniques
    
**2. Background**

    2.1 Finite Difference Methods
    2.2 Chain Rule
    2.3 Computational Graph Structure
    2.4 Dual Numbers
    2.5 Elementary Functions
    
**3. Package Usage**

    3.1 User Interaction
    3.2 Importing AD20
    3.3 Instantiating AD20 Objects
    
**4. Software Organization**

    4.1 Directory Structure
    4.2 Modules and Functionality
    4.3 Testing and Coverage
    4.4 Package Distribution
    
**5. Implementation**

    5.1 Core Data Structures
    5.2 Classes
    5.3 Class Methods and Attributes
    5.4 External Dependencies
    5.5 Elementary Functions

# 1. Introduction
The AD20 package performs the forward mode of automatic differentiation of user defined functions, evaluating both the function and its derivatives to machine precision.

## 1.1 Automatic Differentiation as a Solution to the Problem of Computing Derivatives
Differentiation is a fundamental operation for computational science, used in a variety of applications from optimization to sensitivity analysis.  For the most meaningful results, the computed derivative must be exact (up to machine precision), rather than just an approximation by a finite difference method, and for the most useful results, the computation of the derivative must be done efficiently, as opposed to symbolic differentiation which requires the build up of expression trees.  To address shortcomings in accuracy and efficiency of numerical differentiation and symbolic differentiation respectively, automatic differentiation is used.

Automatic differentiation (AD) is a method of computing derivatives of a function or computer program, based on the principle that any function (or program) is composed of elementary operations and elementary functions.  The derivatives of these elementary operations and elementary functions are easily defined and computed.  By repeatedly applying the chain rule, automatic differentiation is able to compute the derivatives of more complex functions, which are formed by the composition of many elementary operations and functions, by an appropriate composition of the derivatives of these elementary operations and functions.  This approach differs and has advantages over two other main techniques for computing derivatives, finite difference methods and symbolic differentiation.

### Not Numerical Differentiation
Numerical differentiation is a technique used to computationally estimate derivatives from the standard definition,

$\frac{df(x)}{dx} = \lim_{h \to 0}\frac{f(x+h)−f(x)}{h}$.

The derivative is approximated by choosing a small nonzero h at which to evaluate the right hand side. Such a technique is easy to code as it requires only defining $f$, but it has limitations in precision due to truncation error and roundoff error, alongside the challenge of choosing an appropriately sized h. More complex finite difference schemes has been proposed to increase the accuracy of the finite difference approximation, requiring more complex programs while still remaining limited as an approximation with sensitivity to the choice of step size.  Unlike numerical differentiation, automatic differentiation does not rely on approximating the derivative through the choice of a small perturbation in the input, and instead computes derivatives exactly to machine precision, thus avoiding these accuracy and stability problems.

### Not Symbolic Differentiation
To address the accuracy and stability shortcomings of numerical differentiation, symbolic differentiation is sometimes used as a method to compute derivatives to machine precision.  However, this technique involves the generation of mathematical expressions of derivatives, relying on complex expression trees.  For complex functions, these expression trees can quickly become very large, making computations inefficient.  By using the chain rule and viewing functions as compositions of basic operations rather than mathematical expressions, automatic differentiation remains accurate up to machine precision but gains efficiency as it does not require the buildup and evaluation of complex expression trees.
 
Thus, it is clear that automatic differentiation has advantages over other commonly used techniques for computing derivatives.  These advantages make the use of AD attractive to many scientific applications. 

## 1.2 Significance of AD Techniques

**This is directly from alexey.radul**
In conclusion, Automatic Differentiation can be used to compute any derivative or gradient, of any function you can program, or of any program that computes a function, with machine accuracy and ideal asymptotic efficiency. The ability to efficiently and accurately compute derivatives is important to many applications, including:

- Real-parameter optimization (many good methods are gradient-based)
- Sensitivity analysis (local sensitivity = ∂(result)/∂(input))
- Physical modeling (forces are derivatives of potentials; equations of motion are derivatives of Lagrangians and Hamiltonians; etc)
- Probabilistic inference (e.g., Hamiltonian Monte Carlo)
- Machine learning

# This is verbatim from lecture 9
# 2. Background

## 2.1 The Finite Difference
Suppose we want to avoid relying on the symbolic computation of the derivative.  An obvious and very convenient way to do so is to use a finite difference.  For a single-variable function, we just write $$\dfrac{\partial f}{\partial x} \approx \dfrac{f\left(x+\epsilon\right) - f\left(x\right)}{\epsilon}$$ for some "small" $\epsilon$.  Let's do a little demo to see how things turn out.

We'll compute the derivative of $$f\left(x\right) = x - \exp\left(-2\sin^{2}\left(4x\right)\right).$$  We've already defined `python` functions for $f\left(x\right)$ and its derivative.  Let's define a function for the first order finite difference.

The finite difference approach is nice because it is quick and easy.  However, it suffers from accuracy and stability problems. 

## 2.2 Chain Rule

### Review of the Chain Rule

At the heart of AD is the famous *chain rule* from calculus.

### Back to the Beginning
Suppose we have a function $h\left(u\left(t\right)\right)$ and we want the derivative of $h$ with respect to $t$.  The derivative is $$\dfrac{\partial h}{\partial t} = \dfrac{\partial h}{\partial u}\dfrac{\partial u}{\partial t}.$$  This is the rule that we used in symbolically computing the derivative of the function $f\left(x\right) = x - \exp\left(-2\sin^{2}\left(4x\right)\right)$ earlier.

### Adding an Argument
Now suppose $h$ has another argument so that we have $h\left(u\left(t\right),v\left(t\right)\right)$.  Once again, we want the derivative of $h$ with respect to $t$.  Applying the chain rule in this case gives
\begin{align}
  \displaystyle 
  \frac{\partial h}{\partial t} = \frac{\partial h}{\partial u}\frac{\partial u}{\partial t} + \frac{\partial h}{\partial v}\frac{\partial v}{\partial t}.
\end{align}

### The Gradient
What if we replace $t$ by a vector $x\in\mathbb{R}^{m}$?  Now we want the gradient of $h$ with respect to $x$.  We write $h = h\left(u\left(x\right),v\left(x\right)\right)$ and the derivative is now 
\begin{align}
  \nabla_{x} h = \frac{\partial h}{\partial u}\nabla u + \frac{\partial h}{\partial v} \nabla v
\end{align}
where we have written $\nabla_{x}$ on the left hand side to avoid any confusion with arguments.  The gradient operator on the right hand side is clearly with respect to $x$ since $u$ and $v$ have no other arguments.

### The General Rule
In general $h = h\left(y\left(x\right)\right)$ where $y\in\mathbb{R}^{n}$ and $x\in\mathbb{R}^{m}$.  Now $h$ is a function of possibly $n$ other functions themselves a function of $m$ variables.  The gradient of $h$ is now given by 
\begin{align}
  \nabla_{x}h = \sum_{i=1}^{n}{\frac{\partial h}{\partial y_{i}}\nabla y_{i}\left(x\right)}.
\end{align}


## 2.3 Graph Structure on Calculations

### The Computational Graph
Consider again the example function $$f\left(x\right) = x - \exp\left(-2\sin^{2}\left(4x\right)\right).$$  We'd like to evalute $f$ at the point $a$.  In the graph, we indicate the input value as $x$ and the output value as $f$.  Note that $x$ should take on whatever value you want it to.

Let's find $f\left(\dfrac{\pi}{16}\right)$.  The evaluation trace looks like:

| Trace    | Elementary Operation                   | Numerical Value                  |
| :------: | :----------------------:               | :------------------------------: |
| $x_{1}$  | $\dfrac{\pi}{16}$                      | $\dfrac{\pi}{16}$                |
| $x_{2}$  | $4x_{1}$                               | $\dfrac{\pi}{4}$                 |
| $x_{3}$  | $\sin\left(x_{2}\right)$               | $\dfrac{\sqrt{2}}{2}$            |
| $x_{4}$  | $x_{3}^{2}$                            | $\dfrac{1}{2}$                   |
| $x_{5}$  | $-2x_{4}$                              | $-1$                             |
| $x_{6}$  | $\exp\left(x_{5}\right)$               | $\dfrac{1}{e}$                   |
| $x_{7}$  | $-x_{6}$                               | $-\dfrac{1}{e}$                  |
| $x_{8}$  | $x_{1} + x_{7}$                        | $\dfrac{\pi}{16} - \dfrac{1}{e}$ |

Of course, the computer holds floating point values.  The value of $f\left(\dfrac{\pi}{16}\right)$ is $-0.17152990032208026$.  We can check this with our function.

One way to visualize what is going on is to represent the evaluation trace with a graph.

![comp-graph](figs/Computational-Graph.png)

## 2.4 Review of Comlex Numbers
Recall that a complex number has the form $$z = a + ib$$ where we *define* the number $i$ so that $i^{2} = -1$.  No real number has this property but it is a useful property for a number to have.  Hence the introduction of complex numbers.  Visually, you can think of a real number as a number lying on a straight line.  Then, we "extend" the real line "up".  The new axis is called the *imaginary* axis.

Complex numbers have several properties that we can use.
* Complex conjugate: $z^{*} = a - ib$.
* Magnitude of a complex number: $\left|z\right|^{2} = zz^{*} = \left(a+ib\right)\left(a-ib\right) = a^{2} + b^{2}$.
* Polar form: $z = \left|z\right|\exp\left(i\theta\right)$ where $\displaystyle \theta = \tan^{-1}\left(\dfrac{b}{a}\right)$.

## 2.5 Dual Numbers
A dual number has a real part and a dual part.  We write $$z = x + \epsilon x^{\prime}$$ and refer to $x^{\prime}$ as the dual part.  We *define* the number $\epsilon$ so that $\epsilon^{2} = 0$.  **This does not mean that $\epsilon$ is zero!**  $\epsilon$ is not a real number.

#### Some properties of dual numbers:
* Conjugate:  $z^{*} = x - \epsilon x^{\prime}$.
* Magnitude: $\left|z\right|^{2} = zz^{*} = \left(x+\epsilon x^{\prime}\right)\left(x-\epsilon x^{\prime}\right) = x^{2}$.
* Polar form: $z = x\left(1 + \dfrac{x^{\prime}}{x}\right)$.

### Example
Recall that the derivative of $y=x^{2}$ is $y^{\prime} = 2xx^{\prime} = 2x$.

Now if we extend $x$ so that it has a real part and a dual part ($x\leftarrow x + \epsilon x^{\prime}$) and evaluate $y$ we have
\begin{align}
  y &= \left(x + \epsilon x^{\prime}\right)^{2} \\
    &= x^{2} + 2xx^{\prime}\epsilon + \underbrace{x^{\prime^{2}}\epsilon^{2}}_{=0} \\
    &= x^{2} + 2xx^{\prime}\epsilon.
\end{align}
#### Notice that the dual part contains the derivative of our function!!

### Example
Evaluate $y = \sin\left(x\right)$ when $x\leftarrow x + \epsilon x^{\prime}$.

We have
\begin{align}
  y & = \sin\left(x + \epsilon x^{\prime}\right) \\
    & = \sin\left(x\right)\cos\left(\epsilon x^{\prime}\right) + \cos\left(x\right)\sin\left(\epsilon x^{\prime}\right).
\end{align}
Expanding $\cos$ and $\sin$ in their Taylor series gives 
\begin{align}
  \sin\left(\epsilon x^{\prime}\right) &= \sum_{n=0}^{\infty}{\left(-1\right)^{n}\dfrac{\left(\epsilon x^{\prime}\right)^{2n+1}}{\left(2n+1\right)!}} = \epsilon x^{\prime} + \dfrac{\left(\epsilon x^{\prime}\right)^{3}}{3!} + \cdots = \epsilon x^{\prime} \\
  \cos\left(\epsilon x^{\prime}\right) &= \sum_{n=0}^{\infty}{\left(-1\right)^{n}\dfrac{\left(\epsilon x^{\prime}\right)^{2n}}{\left(2n\right)!}} = 1 + \dfrac{\left(\epsilon x^{\prime}\right)^{2}}{2} + \cdots = 1.
\end{align}
Note that the definition of $\epsilon$ was used which resulted in the collapsed sum.

So we see that 
\begin{align}
  y & = \sin\left(x\right) + \cos\left(x\right) x^{\prime} \epsilon.
\end{align}
And once again the real component is the function and the dual component is the derivative.


## 2.6 Elementary Functions
Any complex equation can be broken into combinations of the elementary functions. Some of those include the elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, tan, sqrt etc.). We will not go into details about how to calculate the derivatives of those functions here, but more information can be found on the following link.

http://www.nabla.hr/FU-DerivativeA5.htm


# 3. Package Usage

## 3.1 User Interaction
Users should use ADnum objects to wrap up all mathematical meaning values and formulas. All operations are processed as an ADnum object. Users need to create an ADnum object for each input variable and use all the mathematical functions defined in the ADmath library to implement special functions.

## 3.2 Importing AD20
	import AD20
or 
	from AD20 import ADnum
	from AD20 import ADmath
	from AD20 import ADgraph


## 3.3 Instantiating AD20
	from AD20 import ADnum
	from AD20 import ADmath
	a = ADnum(2)
	b = ADmath.sin(a)
	
Both a and b are ADnum objects, which have the attributes described in the class implementation below.


# 4. Software Organization
We would like to let the user use all numerical operations defined in our AD20 package. Within AD20 package, there is ADnum module, ADmath module  and ADgraph module

For either a scalar or vector input (either as a numpy array or a list), we will convert the input into an ADnum object, which can interact with the other modules. ADnum will also contain an overloaded version of basic operations, including addition, subtraction, multiplication, division, and exponentiation, so that the value and derivative are correctly updated.

For special functions, we will use ADmath to compute the numerical values and the corresponding derivatives. In particular, ADmath will contain functions abs, exp, log, sin, cos, and tan.

To show a calculation graph, we use ADgraph (and ADtable) to show the forward mode calculation process.

###  4.1 Directory Structure
    AD20/
        AD20/
            __init__.py
                ADnum/
                    __init__.py
                    ADnum.py
                ADmath/
                    __init__.py
                    ADmath.py
                ADgraph/
                    __init__.py
                    ADgraph.py
                    ADtable.py
        Tests/
            __init__.py
            test_AD20.py
    README.md
    setup.py
    LICENSE

###  4.2 Modules and Functionality
ADnum: wrap numbers or tuples as a AD object. Moreover, do all of the numerical operations and keep track of all derivatives
ADmath: assign special math meanings and functions to ADnum’s and keep track of the derivatives
ADgraph: trace the calculation process and generate table or graph

In particular, these modules contain the following:
ADnum.py contains the class for ADnum.  This class is fully described below.  It takes as input a single scalar input or a vector input (as either a numpy array or list) and outputs an ADnum object.  Within this class, we will overload basic operations as outlined below.

###  4.3 Testing and Coverage
The tests will be stored in the tests directory (see the repo structure above).  We will use pytest to perform our testing, using TravisCI and Coveralls for continuous integration and verifying code coverage respectively.

###  4.4 Package Distribution
We will use PIP in PyPi to distribute our package.

# 5. Implementation
Automatic differentiation will be implemented through the use of ADnum objects and building the functions for which we want to take derivatives from these ADnum objects as well as the special functions defined for ADnum objects in the ADmath module.  Each of these functions is itself an ADnum object so has an associated value and derivative which was updated when constructing the ADnum object through basic operations and special functions.

### 5.1 Core Data Structures
The main data structure used to represent the functions on which we are performing automatic differentiation will be tuples, with the first entry the value of the ADnum object and the second entry its derivative.  In the case of scalar input, the derivative is also a float.  For vector valued input, the derivative is the gradient of the function, stored as a numpy array.
In order to build and store computational graphs, we will use a dictionary as the computational graph, where the keys are the nodes of the graph, stored as ADnum objects, and the values associated with each key are the children of that node, stored as lists of ADnum objects.

### 5.2 Implemented Classes
The main class will be implemented in the ADnum module, which will create ADnum objects.  The ADnum objects will store the current value of the function and its derivative as attributes.  By combining simple ADnum objects with basic operations and simple functions, we can construct any function we like.  For example,

    X = AD20.ADnum(4)
    Y = AD20.ADnum(0)
    F = X+ADmath.sin(Y)
    
Where F is now an ADnum object, and ADmath.sin() is a specially defined sine function which takes as input an ADnum object and returns an ADnum object, which allows us to evaluate F and its derivative,

    F.val = 4
    F.deriv = [1, 1] 
    X.val = 4
    X.deriv = 1

In addition to the sine function, the ADmath module will also implement the other trigonometric functions, the natural exponential, and the natural logarithm.

We will also implement a class, ADgraph, for computational graphs.  The constructor takes as input a dictionary, as described above where the keys are nodes and values are the children of the key node. 	This can then be used to perform forward propagation and could be extended later to include back propagation as an extension of our project.
 
### 5.3 Class Methods and Attributes
Each ADnum object will have two attributes for the two major functions desired of the class.  The val attribute will be the ADnum object evaluated at the given value and the der attribute will be its derivative.  In addition, each ADnum object will have a graph attribute, which stores the dictionary which can be used to build a computational graph in the ADgraph class.  The ADnum class will also include methods to overload basic operations, __add__(), __radd__(), __mul__(), __rmul__(), __sub__(), __truedivide__(), and __pow__().  The result of overloading is that the adding, subtracting, multiplying, dividing, or exponentiating two ADnum objects returns an ADnum object as well as addition or multiplication by a constant.  For example, Y1, Y2, and Y3 would all be recognized as ADnum objects:

    X1= ADnum(7)
    X2 = ADnum(15)
    Y1 = X1+X2
    Y2 = X1*X2+X1
    Y3 = 5*X1+X2+100

The resulting ADnum objects have both a value and derivative.

The ADgraph class will be constructed from a dictionary, stored in the attribute dict.  This class will also have an attribute inputs, which stores the nodes which have no parents.  This class will implement a deriv method which returns the derivative from the computational graph.

### 5.4 External Dependencies
In order to implement the elementary functions, our class will rely on numpy’s implementation of the trigonometric functions, exponential functions, and natural logarithms for evaluation of these special functions.

We will also use numpy to implement matrix and vector multiplication in cases where the function is either vector valued or takes a vector as an input.

### 5.5 Elementary Functions
As outlined above, we will have a special ADmath module which defines the trigonometric, exponential, and logarithmic functions to be used on ADnum objects, so that they both take as input and return an ADnum object.